Changes made by @fhalamos from repo https://github.com/jmather625/predicting-poverty-replication

Make sure you download the 2016 Household LSMS survey data for Malawi, Ethiopia, and Nigeria from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/`. Malawi's data should be named `malawi_2016/LSMS`, Ethiopia's should be named `ethiopia_2015/LSMS`, and Nigeria's should be named `nigeria_2015/LSMS`. Nightlights data should be downloaded from https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html using the annual composite from 2015 in tile 2 and tile 5.

In [ ]:
! pip install geoio
! pip install utils

In [1]:
import pandas as pd
import numpy as np
import os
import geoio

In [ ]:
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls '/content/drive/MyDrive/Data Science GRDS [Felipe]/Poverty Measurement/predicting-poverty-replication'

activation_maps  figures	LICENSE  README.md	   scripts
data		 gold_standard	papers	 requirements.txt  utilities


In [4]:
BASE_DIR = '/content/drive/MyDrive/Data Science GRDS [Felipe]/Poverty Measurement/predicting-poverty-replication'

In [6]:
NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif'),
                    os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_75N060W.tif')]

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [9]:
import sys
sys.path.append(BASE_DIR)
from utilities import create_space

In [12]:
def process_country_data(country_abbreviation, lsms_dir, consumption_file, hhsize_col, geovariables_file, lat_col, lon_col, ppp, hh_id_col, consumption_ph_col=None, consumption_pc_col=None):
    '''
    consumption_ph_col # per household
    consumption_pc_col # per capita
    '''
    #Check files exist
    for file in [consumption_file, geovariables_file]:       
        file_path = os.path.join(lsms_dir, file)
        assert os.path.isfile(file_path), print(f'Could not find {file_path}')
    
    #Load consumption df    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    
    #Create consumption per household column
    if consumption_ph_col is not None:
      df['cons_ph'] = df[consumption_ph_col]
    #If consumption was reported per capita, multiply by hh size to get per household
    elif consumption_pc_col is not None:
      df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]

    #Create people per household column
    df['pph'] = df[hhsize_col]

    # Create consumption per household adjusted pp
    df['cons_ph'] = df['cons_ph'] / ppp / 365

    #Filter df to only selected columns
    df = df[[hh_id_col, 'cons_ph', 'pph']]

    #Load geovariables df
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[[hh_id_col, lat_col, lon_col]]

    #Rename lat lon columns after cluster
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)

    #Merge geodata with consumption data on hh identifier
    df_combined = pd.merge(df, df_cords, on=hh_id_col)

    #Drop repeted identifier
    df_combined.drop([hh_id_col], axis=1, inplace=True)

    #Drop na values
    df_combined.dropna(inplace=True)

    #Aggregate consumption at the cluster level
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()

    #Compute consumption per capita
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = country_abbreviation
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]


In [13]:
#Countries parameters
countries_params = {}

#Malawi parameters
country_params = {}
country_params['lsms_dir'] = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
country_params['consumption_file'] = os.path.join('consumption_aggregate','ihs4 consumption aggregate.csv')
country_params['consumption_pc_col'] = None
country_params['consumption_ph_col'] = 'rexpagg' # per household
country_params['hhsize_col'] = 'hhsize' # people in household
country_params['geovariables_file'] = os.path.join('household_geovariables','householdgeovariablesihs4.csv')
country_params['lat_col'] = 'lat_modified'
country_params['lon_col'] = 'lon_modified'
country_params['ppp'] = 215.182 # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
country_params['hh_id_col'] = 'case_id'
country_params['country_abbreviation'] = 'mw'
countries_params['mw'] = country_params

#Ethiopia parameters
country_params = {}
country_params['lsms_dir'] = os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'LSMS')
country_params['consumption_file'] = 'Consumption Aggregate/cons_agg_w3.csv'
country_params['consumption_pc_col'] = 'total_cons_ann'
country_params['consumption_ph_col'] = None
country_params['hhsize_col'] = 'hh_size' # people in household
country_params['geovariables_file'] = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
country_params['lat_col'] = 'lat_dd_mod'
country_params['lon_col'] = 'lon_dd_mod'
country_params['ppp'] = 7.882 # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
country_params['hh_id_col'] = 'household_id2'
country_params['country_abbreviation'] = 'eth'
countries_params['eth'] = country_params

#Nigeria parameters
country_params = {}
country_params['lsms_dir'] = os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'LSMS')
country_params['consumption_file'] = 'cons_agg_wave3_visit1.csv'
country_params['consumption_pc_col'] = 'totcons' # per capita
country_params['consumption_ph_col'] = None
country_params['hhsize_col'] = 'hhsize' # people in household
country_params['geovariables_file'] = 'nga_householdgeovars_y3.csv'
country_params['lat_col'] = 'LAT_DD_MOD'
country_params['lon_col'] = 'LON_DD_MOD'
country_params['ppp'] = 95.255 # purchasing power parity for nigeria in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=NG)
country_params['hh_id_col'] = 'hhid'
country_params['country_abbreviation'] = 'ng'
countries_params['ng'] = country_params

In [18]:
#Load countries data

countries_dfs = {}

for country_abbrv in ['mw', 'eth', 'ng']:
    countries_dfs[country_abbrv] = process_country_data(
        country_abbreviation = country_abbrv,
        lsms_dir = countries_params[country_abbrv]['lsms_dir'],
        consumption_file = countries_params[country_abbrv]['consumption_file'],
        hhsize_col = countries_params[country_abbrv]['hhsize_col'],
        geovariables_file = countries_params[country_abbrv]['geovariables_file'],
        lat_col = countries_params[country_abbrv]['lat_col'],
        lon_col = countries_params[country_abbrv]['lon_col'],
        ppp = countries_params[country_abbrv]['ppp'],
        hh_id_col = countries_params[country_abbrv]['hh_id_col'],
        consumption_ph_col=countries_params[country_abbrv]['consumption_ph_col'],
        consumption_pc_col=countries_params[country_abbrv]['consumption_pc_col'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [29]:
def get_nightlights(df, tif):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    '''

    tif_array = np.squeeze(tif.get_data())

    cluster_nightlights = []
    for i,r in df.iterrows():

        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon, s=10)

        #Find pixels for given lat lon pairs
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)

        #Validate values obtained for space is inside tif
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        
        #Parse pixel values to integers
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        
        #Compute mean nightlight
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    return cluster_nightlights

In [38]:
#Add nightlights data to each cluster

#We knew ahead of time the 0th tif is for Malawi, and the 1st tif is for Ethiopia and Nigeria
for (country_abbrv, tif_id) in [('mw',0),('eth',1),('ng',1)]:
  print(f'Computing nightlights for {country_abbrv}')
  countries_dfs[country_abbrv]['nightlights'] = get_nightlights(countries_dfs[country_abbrv], tifs[tif_id])
  print(f'Finished {country_abbrv}')

Computing nightlights for mw
Finished mw
Computing nightlights for eth
Finished eth
Computing nightlights for ng
Finished ng


In [34]:
#Not sure if this is necessary any more
# import gc
# gc.collect()
# import psutil
# psutil.virtual_memory()

In [39]:
#Save processed data
for (country_abbrv, country_folder) in [('mw','malawi_2016'), ('eth','ethiopia_2015'), ('ng','nigeria_2015')]:
    #Create folder for processed data
    os.makedirs(os.path.join(COUNTRIES_DIR, country_folder, 'processed'), exist_ok=True)
    
    #Save df to csv
    countries_dfs[country_abbrv].to_csv(os.path.join(COUNTRIES_DIR, country_folder, 'processed/clusters.csv'), index=False)
